**Imports**

In [3]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

import pandas as pd

from pymongo import MongoClient
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import time
import pprint

from datetime import timedelta, date
import datetime

import numpy as np
import matplotlib as plt

#### Initiating Selenium, signing in, navigating to page

In [ ]:
webdriver = "/Users/kathrynjohnson/Downloads/chromedriver"

driver = Chrome(webdriver)

login_url = "https://myaccount.nytimes.com/auth/login"

driver.get(login_url)

html_source = driver.page_source

In [ ]:
request_url = 'https://www.nytimes.com/crosswords'

driver.get(request_url)

html_source = driver.page_source

#### Template Code

In [ ]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

puzzle_dict = {}
start_date = date(2019, 1, 1)
end_date = date(2019, 1, 2)

for single_date in daterange(start_date, end_date):
    #tell selenium to access the site and wait for it to load
    url = "https://www.nytimes.com/crosswords/game/daily/" + single_date.strftime("%Y/%m/%d") + "/"
    driver.get(url)
    time.sleep(10)
    
    #get all the clues and return them in a list
    clue_element = driver.find_elements_by_class_name("Clue-text--3lZl7")
    clue = [x.text for x in clue_element]
    
    #get the day of the week from the puzzle details
    day_element = driver.find_elements_by_class_name('PuzzleDetails-date--1HNzj')
    for x in day_element:
        date_info = x.text
        date_split = date_info.split(' ')
        day_of_week = date_split[0]
    author_element = driver.find_elements_by_class_name("PuzzleDetails-byline--16J5w")
    
    #get the author of the puzzle, parsing will be based on whether puzzle has a special title
    for x in author_element:
        if x.text[0] is 'T':
            byline = x.text[16:]
            byline_split = byline.split('Edited')
            author = byline_split[0]
        elif x.text[0] is '"':
            byline = x.text.replace('"', ' ')
            byline2 = byline.replace('Edited ', ' ')
            byline3 = byline2.replace('by', 'By')
            byline_split = byline3.split('By ')
            author = byline_split[1]
    
    #get the solve time and format it
    timer_element = driver.find_elements_by_class_name('timer-count')
    for x in timer_element:
        if len(x.text) == 4:
            timer = '00:0' + x.text
        elif len(x.text) == 5:
            timer = '00:' + x.text
        elif len(x.text) == 7:
            timer = '0' + x.text
        else:
            timer = x.text
    
    #convert timer to seconds
    date_time = datetime.datetime.strptime(timer, "%H:%M:%S")
    a_timedelta = date_time - datetime.datetime(1900, 1, 1)
    seconds = a_timedelta.total_seconds()
    
    #return all of the data in dictionary with the puzzle date as the key
    puzzle_dict[single_date.strftime("%Y-%m-%d")] = (seconds, day_of_week, author, clue)
    
    #tell selenium to wait a few more seconds before the next pull
    time.sleep(5)

In [ ]:
df = pd.DataFrame.from_dict(puzzle_dict, orient='index', columns=['solve time(seconds)', 'day', 'author', 'clues'])

#### Separating Days of the Week

In [ ]:
df_Sun = df[df['day'] == 'Sunday']
df_Mon = df[df['day'] == 'Monday']
df_Tues = df[df['day'] == 'Tuesday']
df_Wed = df[df['day'] == 'Wednesday']
df_Thurs = df[df['day'] == 'Thursday']
df_Fri = df[df['day'] == 'Friday']
df_Sat = df[df['day'] == 'Saturday']